In [1]:
import itertools
from parlay_system import *
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1500)


In [21]:
tott = MoneyLine(event="tott", bet_amount=100, odds=-186)
tott_bright = MoneyLine(event="TIEbright", bet_amount=100, odds=340)
bright = MoneyLine(event="bright", bet_amount=100, odds=480)

bourn = MoneyLine(event="bourn", bet_amount=100, odds=250)
bourn_arse = MoneyLine(event="TIEbourn", bet_amount=100, odds=290)
arse = MoneyLine(event="arse", bet_amount=100, odds=-103)

crys = MoneyLine(event="crys", bet_amount=100, odds=128)
crys_westham = MoneyLine(event="TIEcrys", bet_amount=100, odds=250)
westham = MoneyLine(event="westham", bet_amount=100, odds=215)

leic = MoneyLine(event="leic", bet_amount=100, odds=240)
leic_liver = MoneyLine(event="TIEleic", bet_amount=100, odds=270)
liver = MoneyLine(event="liver", bet_amount=100, odds=108)

newcast = MoneyLine(event="newcast", bet_amount=100, odds=220)
newcast_ever = MoneyLine(event="TIEnewcast", bet_amount=100, odds=240)
ever = MoneyLine(event="ever", bet_amount=100, odds=125)

trinaries = [
#     [tott, tott_bright, bright],
#     [bourn, bourn_arse, arse],
    [crys, crys_westham, westham],
    [leic, leic_liver, liver],
    [newcast, newcast_ever, ever]
]

results = [
#     [0, 1, 0],
#     [1, 0, 0],
    [1, 0, 0],
    [1, 0, 0],
    [1, 0, 0]
]

override_arr = [0, 0, 99]

In [22]:
all_subsets = list(itertools.product([0, 1, 2], [0, 1, 2]))
all_subsets = [(0, 1, 2)]


In [23]:
def create_dicts(in_binaries, in_results):
    ml_lookup = {}
    ml_win_lookup = {}
    id = 0
    for bi in range(len(in_binaries)):
        binary = in_binaries[bi]
        for ml_i in range(len(binary)):
            ml = binary[ml_i]
            ml.set_index(id) 
            is_winner = in_results[bi][ml_i]
            ml_lookup[id] = ml.event
            ml_win_lookup[id] = is_winner
            id += 1
    
    return ml_lookup, ml_win_lookup

ml_lookup, ml_outcome_lookup = create_dicts(trinaries, results)

In [24]:
csv = []
df = pd.DataFrame()

for i in range(len(all_subsets)):
    current_subset = all_subsets[i]
    parlay_binaries = []

    for j in range(len(current_subset)):
        index = current_subset[j]
        parlay_binaries.append(trinaries[j])

        ps = ParlaySystem(binaries=parlay_binaries,
                    target_profit=0.5,
                    bounds=(0.005, 30),
                    binary_index_arr=list(current_subset),
                    binary_results_arr=results,
                    index_to_ml=ml_lookup,
                    index_to_outcome=ml_outcome_lookup,
                    override_arr=override_arr
                    )
    
    csv_res, raw_df = ps.slsqp_solver()
    df = pd.concat([df, raw_df])
    csv.append(csv_res)

     fun: -0.06566043956795148
     jac: array([ 8.12444445e-02,  2.38222219e-02,  3.54000000e-01,  1.77778304e-04,
       -6.23111110e-02,  2.97000000e-01,  4.37937778e-01,  4.02808889e-01,
        6.04800000e-01, -4.10370370e-01, -4.98518518e-01,  8.33333377e-03,
       -5.34814815e-01, -6.30740740e-01, -7.91666666e-02,  1.37185186e-01,
        8.32592594e-02,  3.93333334e-01, -2.69333333e-01, -3.48666666e-01,
        1.07500000e-01, -3.81333333e-01, -4.67666667e-01,  2.87500005e-02,
        2.23466667e-01,  1.74933333e-01,  4.54000000e-01])
 message: 'Positive directional derivative for linesearch'
    nfev: 383
     nit: 16
    njev: 12
  status: 8
 success: False
       x: array([0.04720892, 0.05084576, 0.02110451, 0.05222185, 0.05106896,
       0.02823402, 0.00797305, 0.01391788, 0.005     , 0.03846587,
       0.03620317, 0.05175462, 0.03534702, 0.03326778, 0.05027131,
       0.04320041, 0.04707304, 0.01540352, 0.04273985, 0.04022575,
       0.04539012, 0.03927446, 0.0369642 , 0.

In [6]:
len(all_subsets)

1

In [7]:
df.describe()

,odds,bet,mult,payout,profit
count,9.000000,9.000000,9.000000,9.000000,9.000000
mean,778.116667,0.005667,8.781167,0.048933,-0.002000
std,227.131044,0.000485,2.271310,0.009284,0.009301
min,413.000000,0.005000,5.130000,0.029700,-0.021000
25%,629.600000,0.005200,7.296000,0.044400,-0.007000
50%,687.500000,0.005800,7.875000,0.048300,-0.003000
75%,971.000000,0.006100,10.710000,0.056000,0.005000
max,1090.000000,0.006100,11.900000,0.059500,0.009000


In [8]:
negative_df = df.loc[df.profit < 0]
negative_df = negative_df.sort_values(by=['ev_stat', 'profit'], ascending=[False, True])
negative_df.describe()

,odds,bet,mult,payout,profit
count,5.000000,5.000000,5.000000,5.000000,5.000000
mean,602.810000,0.006040,7.028100,0.042580,-0.008400
std,110.898075,0.000134,1.108981,0.007522,0.007403
min,413.000000,0.005800,5.130000,0.029700,-0.021000
25%,608.750000,0.006100,7.087500,0.043000,-0.008000
50%,629.600000,0.006100,7.296000,0.044400,-0.007000
75%,675.200000,0.006100,7.752000,0.047500,-0.003000
max,687.500000,0.006100,7.875000,0.048300,-0.003000


In [9]:
positive_df = df.loc[df.profit >= 0]
positive_df = positive_df.sort_values(by=['ev_stat', 'profit'], ascending=[False, False])
positive_df.describe()

,odds,bet,mult,payout,profit
count,4.00000,4.000000,4.000000,4.000000,4.000
mean,997.25000,0.005200,10.972500,0.056875,0.006
std,76.97348,0.000283,0.769735,0.001750,0.002
min,908.00000,0.005000,10.080000,0.056000,0.005
25%,955.25000,0.005000,10.552500,0.056000,0.005
50%,995.50000,0.005100,10.955000,0.056000,0.005
75%,1037.50000,0.005300,11.375000,0.056875,0.006
max,1090.00000,0.005600,11.900000,0.059500,0.009


In [10]:
negative_df.head(200)

,event,index[],result,ev_stat,odds,bet,mult,payout,profit
0,crys_newcast,"[0, 3]","[1, 1]",True,629.60,0.0061,7.2960,0.0444,-0.007
2,crys_ever,"[0, 5]","[1, 0]",False,413.00,0.0058,5.1300,0.0297,-0.021
8,westham_ever,"[2, 5]","[0, 0]",False,608.75,0.0061,7.0875,0.0430,-0.008
1,crys_TIEnewcast,"[0, 4]","[1, 0]",False,675.20,0.0061,7.7520,0.0475,-0.003
5,TIEcrys_ever,"[1, 5]","[0, 0]",False,687.50,0.0061,7.8750,0.0483,-0.003


In [11]:
positive_df.head(200)

,event,index[],result,ev_stat,odds,bet,mult,payout,profit
4,TIEcrys_TIEnewcast,"[1, 4]","[0, 0]",False,1090.0,0.0050,11.90,0.0595,0.009
3,TIEcrys_newcast,"[1, 3]","[0, 1]",False,1020.0,0.0050,11.20,0.0560,0.005
6,westham_newcast,"[2, 3]","[0, 1]",False,908.0,0.0056,10.08,0.0560,0.005
7,westham_TIEnewcast,"[2, 4]","[0, 0]",False,971.0,0.0052,10.71,0.0560,0.005


In [12]:
positive_df.groupby('ev_stat').count()

,event,index[],result,odds,bet,mult,payout,profit
ev_stat,,,,,,,,
False,4,4,4,4,4,4,4,4


In [13]:
negative_df.groupby('ev_stat').count()

,event,index[],result,odds,bet,mult,payout,profit
ev_stat,,,,,,,,
False,4,4,4,4,4,4,4,4
True,1,1,1,1,1,1,1,1


In [14]:
neg = negative_df.reset_index(drop=True).style.applymap(color_negative_red)
neg

,event,index[],result,ev_stat,odds,bet,mult,payout,profit
0,crys_newcast,"['0', '3']","[1, 1]",True,629.6,0.0061,7.296,0.0444,-0.007
1,crys_ever,"['0', '5']","[1, 0]",False,413,0.0058,5.13,0.0297,-0.021
2,westham_ever,"['2', '5']","[0, 0]",False,608.75,0.0061,7.0875,0.043,-0.008
3,crys_TIEnewcast,"['0', '4']","[1, 0]",False,675.2,0.0061,7.752,0.0475,-0.003
4,TIEcrys_ever,"['1', '5']","[0, 0]",False,687.5,0.0061,7.875,0.0483,-0.003


In [15]:
pos = positive_df.reset_index(drop=True).style.applymap(color_positive_green)
pos

,event,index[],result,ev_stat,odds,bet,mult,payout,profit
0,TIEcrys_TIEnewcast,"['1', '4']","[0, 0]",False,1090,0.005,11.9,0.0595,0.009
1,TIEcrys_newcast,"['1', '3']","[0, 1]",False,1020,0.005,11.2,0.056,0.005
2,westham_newcast,"['2', '3']","[0, 1]",False,908,0.0056,10.08,0.056,0.005
3,westham_TIEnewcast,"['2', '4']","[0, 0]",False,971,0.0052,10.71,0.056,0.005


In [16]:
sf49ers = MoneyLine(event="parlay", bet_amount=100, odds=1002)
sf49ers.print_stats()

    event  bet_amount  odds  multiplier  payout
0  parlay         100  1002       11.02  1102.0


In [17]:
pos = MoneyLine(event="pos", bet_amount=100, odds=-1020)
pos.print_stats()

  event  bet_amount  odds  multiplier      payout
0   pos         100 -1020    1.098039  109.803922


In [18]:
neg = MoneyLine(event="neg", bet_amount=1.0, odds=-305)
neg.print_stats()

  event  bet_amount  odds  multiplier    payout
0   neg         1.0  -305    1.327869  1.327869


In [19]:
neg = MoneyLine(event="neg", bet_amount=1.0, odds=-285)
neg.print_stats()

  event  bet_amount  odds  multiplier    payout
0   neg         1.0  -285    1.350877  1.350877
